# Dynamic scope in Python

When we write something like:

```python
def f():
    with decimal.localcontext(ctx1):
        g()
    
def g():
    # decimal operations here use ctx1 (or whatever the caller set)
    a + b
    with decimal.localcontext(ct2):
        # decimal operations here use ctx2 (no matter what the caller did)
        a + b
    # decimal operations here use ctx1 again (or whatever the caller set)
    a + b
```

then conceptually we're using [dynamic scoping](https://en.wikipedia.org/wiki/Scope_%28computer_science%29#Dynamic_scoping): you can think of it as, whenever you do a decimal operation, and the decimal module needs to find the current decimal context, then it walks up the callstack until it finds the first enclosing `with decimal.localcontext(...)`, and uses that. (Of course the actual implementation is different, but from the user point of view, this is the illusion it's trying to maintain.)

Crucially, this is different than Python's normal scoping rules. Normal variable lookup in Python uses "static scoping": if it can't find a variable locally, then it looks around in the enclosing *file*, not in the *runtime caller*. 99% of the time this is what you want. But the other 1% of the time you have something like the decimal context, or numpy's errstate, or the flask request object, and then dynamic scoping is the thing. But given how rich Python's semantics are – especially around generators – it turns out that right now it's almost impossible to actually implement dynamic scoping properly in Python code.

But there is one example of proper dynamic scoping already built into the interpreter: `sys.exc_info`, which you can access inside an `except:` or `finally:` block to get information about the exception that's currently being handled. Let's walk through some examples to see how the interpreter handles `sys.exc_info`, and why we currently can't make the decimal context work similarly.

## exc_info basics

First, import `sys`, we'll obviously need that to call `sys.exc_info`.

In [28]:
import sys

Now, the simplest example of `sys.exc_info`: outside of any `except`/`finally` blocks, it's `(None, None, None)`. Inside an `except` block, it contains information about the exception being handled:

In [29]:
print("no exceptions in progress:", sys.exc_info())
try:
    raise ValueError
except:
    # exc_info contains the ValueError
    print("inside except: block:", sys.exc_info())
# exc_info is (None, None, None)
print("after except: block:", sys.exc_info())

no exceptions in progress: (None, None, None)
inside except: block: (<class 'ValueError'>, ValueError(), <traceback object at 0x7f9de8296108>)
after except: block: (None, None, None)


The call to `sys.exc_info` doesn't have to be *directly* inside the `except` block – if you call it from a subroutine, then it "walks up the call stack" to find the first `except` block, and prints information about it:

In [30]:
def subroutine():
    print("in subroutine:", sys.exc_info())

try:
    raise ValueError
except:
    subroutine()

in subroutine: (<class 'ValueError'>, ValueError(), <traceback object at 0x7f9de82a9b08>)


This is great, this is what we want. For example, if our subroutine raises an exception, this feature of `sys.exc_info` is what allows [implicit exception chaining](https://blog.ionelmc.ro/2014/08/03/the-most-underrated-feature-in-python-3/) to work.

Of course, so far this could be faked by a simple global variable. Like maybe secretly `except` does something like:

```python
except:
    # invisible line inserted by the interpreter
    sys._exc_info = (..., ..., ...)
    # then your code here
    ...
```

And in Python 2 I think that's basically how it worked (I'm not sure of the details). But Python 3 is more clever. Consider what happens if we one `except` block nested inside another:

In [31]:
try:
    raise ValueError
except:
    # exc_info contains the ValueError
    print("single-nested:", sys.exc_info())
    try:
        raise KeyError
    except:
        # exc_info contains the KeyError
        print("double-nested:", sys.exc_info())
    # exc_info contains the ValueError again
    print("single-nested again:", sys.exc_info())


single-nested: (<class 'ValueError'>, ValueError(), <traceback object at 0x7f9de8f43688>)
double-nested: (<class 'KeyError'>, KeyError(), <traceback object at 0x7f9de8f436c8>)
single-nested again: (<class 'ValueError'>, ValueError(), <traceback object at 0x7f9de8f43688>)


In particular, notice how when we exit the inner `except` block, `sys.exc_info` switches *back* to showing `ValueError`. If we want to fake this, then a simple global variable isn't enough: the inner `except` block needs to save the old `exc_info` somewhere, and then restore it again on exit. Something like:

```python
except:
    # invisible lines inserted by the interpreter:
    _tmp_exc_info_0 = sys._exc_info
    sys._exc_info = (..., ..., ...)
    # then your code here:
    ...
    # then another invisible line inserted by the interpreter:
    sys._exc_info = _tmp_exc_info_0
```

And this save/restore pattern is exactly how Python-level context managers like `decimal.localcontext` work today.

But: it's not how `sys.exc_info` works! To see the special magic, we need to look at how `exc_info` and generators interact.

## generators reveal exc_info's dynamic scoping magic

First let's look at a pretty simple extension of mixing `exc_info` and generators. We'll make a generator that accesses `exc_info`, and iterate it from different contexts. This isn't really showing anything new and doesn't require any new magic, it's just demonstrating that our `subroutine` example from above still works when `subroutine` is replaced by a generator. This could still be faked using the save/restore trick:

In [32]:
def gen():
    while True:
        print("inside gen:", sys.exc_info())
        yield

g = gen()
try:
    raise ValueError
except:
    # Iterate with ValueError, the generator sees ValueError
    next(g)

# Iterate with no exception, the generator sees no exception
next(g)

try:
    raise KeyError
except:
    # Iterate with KeyError, the generator sees KeyError
    next(g)

inside gen: (<class 'ValueError'>, ValueError(), <traceback object at 0x7f9de8289a88>)
inside gen: (None, None, None)
inside gen: (<class 'KeyError'>, KeyError(), <traceback object at 0x7f9de8289a88>)


So far so good. Now let's see a trickier case, where we suspend a generator *inside* an `except` block. This is a case where real dynamic scoping and the save/restore trick act differently.

If `exc_info` has real dynamic scoping, then `sys.exc_info()` calls outside the generator should never see the `RuntimeError`, because that's only visible inside the generator. And the generator's `sys.exc_info()` call should always describe to the `RuntimeError`, because that's the closest enclosing block, even if the next frame out – the one that's calling `next` — keeps changing.

On the other hand, if `exc_info` is using the save/restore trick – like `decimal.localcontext` does – then `sys.exc_info()` should always refer to the *last exception that happened*, no matter where it is in the program. So the first time we suspend the generator, we should be able to see the `RuntimeError` in the calling frame. And if we call `next` to re-enter the generator with some other exception active, then the `sys.exc_info()` call inside the generator will see that other exception, rather than the `RuntimeError`.

So... what happens?

In [33]:
def gen():
    try:
        raise RuntimeError("raise inside gen")
    except:
        # This loop is identical to our previous generator.
        # Just, now it's embedded inside an except block.
        while True:
            # If this always prints RuntimeError, that shows we have real dynamic scoping.
            print("inside gen:", sys.exc_info())
            yield
            
g = gen()
print("outside gen, before first iteration:", sys.exc_info())
next(g)
# If this prints '(None, None, None)', that shows we have real dynamic scoping
# (The generator's exception can't "leak out" into it's calling context.)
print("outside gen, after first iteration:", sys.exc_info())

try:
    raise ValueError
except:
    print("\noutside gen:", sys.exc_info())
    next(g)

outside gen, before first iteration: (None, None, None)
inside gen: (<class 'RuntimeError'>, RuntimeError('raise inside gen',), <traceback object at 0x7f9de82afa48>)
outside gen, after first iteration: (None, None, None)

outside gen: (<class 'ValueError'>, ValueError(), <traceback object at 0x7f9de82af9c8>)
inside gen: (<class 'RuntimeError'>, RuntimeError('raise inside gen',), <traceback object at 0x7f9de82afa48>)


Inside the generator we always see the `RuntimeError`, and outside the generator we never see the `RuntimeError`. So `exc_info` must be using real dynamic scoping.

# Conclusion and takeaways

First, this is the example that currently, `decimal.localcontext` and friends *cannot get right*. You cannot fake this with a thread-local + saving/restoring values, so right now there is no way to do what `exc_info` does in pure Python. Which is problematic, because if you think about it, this last example is something that comes up all the time async/await code: all it requires is that you use `await` inside an `except` or `finally` block. If `exc_info` didn't handle this case correctly, then async/await wouldn't work at all. But if you use `await` inside a `decimal.localcontext` block, that doesn't work. Fortunately people don't notice so much, because `decimal.localcontext` is much rarer than exceptions. But fixing this is what motivates PEP 550.

But, here's the problem: the first draft of PEP 550 basically adds the concept of "generator local" variables, as a refinement of "thread local variables". This allows you to get the behavior in the second generator example. But what it *can't* do is give us the behavior in the *first* generator example. If we used PEP 550 context to store `exc_info`, then it would be set once when we first started the generator, and then wouldn't change after that – it couldn't "see" the calling context. And this would be bad because it, for example, breaks implicit exception chaining.

Maybe PEP 550's goal should be to allow pure Python code to implement real dynamic scoping, like `exc_info` does.

# Bonus appendix

Actually, `exc_info`'s dynamic scoping is a bit buggy in edge cases. Here's a version of the single/double-nested example from above, but now we've factored out the inner `try`/`except` into a generator. It should work the same... but it doesn't; the exception context gets lost:

In [34]:
def gen():
    try:
        raise KeyError
    except:
        yield
        print("double-nested:", sys.exc_info())
    # This should print ValueError, but it doesn't!
    print("single-nested again:", sys.exc_info())

g = gen()
next(g)
try:
    raise ValueError
except:
    print("single-nested:", sys.exc_info())
    next(g, None)

single-nested: (<class 'ValueError'>, ValueError(), <traceback object at 0x7f9de829ddc8>)
double-nested: (<class 'KeyError'>, KeyError(), <traceback object at 0x7f9de829d0c8>)
single-nested again: (None, None, None)


(This is a variation of a [bug found by Armin Rigo](https://bugs.python.org/issue28884#msg282532).)

Maybe "real" dynamic scoping support would help fix this kind of thing too?